## Main code for Kaggle - Optiver Realized Volatility Prediction
@LaurentMombaerts 13/07/2021

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


**Lib Import / Data loading**

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import glob
from sklearn.metrics import r2_score

# Models
from persistence import *

# Load dataset
# pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv') # Uncomment this for online submission 
train = pd.read_csv(r'C:\Users\laurent\ownCloud\Data\Kaggle\optiver-realized-volatility-prediction\train.csv') # Comment the local path for online submission
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]

**Functions**

In [8]:
# Competition metric
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

# Prediction function (chose here which prediction strategy to use)
def prediction_function(files,column_name):
    
    # Naive prediction (persistence model)
    prediction = past_realized_volatility_per_stock(list_file=files,prediction_column_name=column_name)
    
    # Option B
    
    return prediction

**Main evaluation code**

In [9]:
# Glob book file train (contains all paths for each file in this folder)
# list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*') # Uncomment this for online submission
list_order_book_file_train = glob.glob(r'C:\Users\laurent\ownCloud\Data\Kaggle\optiver-realized-volatility-prediction\book_train.parquet\*') # Comment the local path for online submission

# Compute predictions
df_past_realized_train = prediction_function(files=list_order_book_file_train,column_name='pred') 

In [10]:
# Merge and evaluate results
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')
print(df_joined.head(5))

# Estimate performances
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)

print('--')
print(f'Performance of prediction: R2 score: {R2}, RMSPE: {RMSPE}')

  row_id    target      pred
0    0-5  0.004136  0.004499
1   0-11  0.001445  0.001204
2   0-16  0.002168  0.002369
3   0-31  0.002195  0.002574
4   0-62  0.001747  0.001894
--
Performance of prediction: R2 score: 0.628, RMSPE: 0.341


**Online submission part**

In [ ]:
# Glob book file test (contains all paths for each file in this folder)
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')

predictions = prediction_function(files=list_order_book_file_test,column_name='target')
predictions.to_csv('submission.csv',index = False)